# medicines

In [21]:
import pandas as pd

# 파일 경로 설정
csv_file_path = '/Users/seungwook/Downloads/약품코드.csv'  # 원본 CSV 파일 경로
cleaned_file_path = '/Users/seungwook/Downloads/약품코드_1차.csv'  # 1차 전처리(제품코드만 있는 CSV 파일 경로)
output_file_path = '/Users/seungwook/Downloads/약품코드_2차.csv'  # 2차 전처리(1차 전처리 파일에서 중복 제거)
sql_file_path = '/Users/seungwook/Downloads/medicines_insert.sql'  # medicines 테이블에 삽입 쿼리

# 1차 전처리: UTF-8 인코딩으로 파일 읽기
df = pd.read_csv(csv_file_path, encoding='utf-8', dtype={'product_code': str})  # product_code를 문자열로 읽음
df_cleaned = df[df['product_code'].notnull()]  # product_code가 null이 아닌 경우만 필터링
df_cleaned.to_csv(cleaned_file_path, index=False, encoding='utf-8')  # 1차 전처리 파일 저장

# 2차 전처리: 중복 제거 후 저장
df = pd.read_csv(cleaned_file_path, encoding='utf-8', dtype={'product_code': str})
df_unique = df.drop_duplicates(subset='product_code')  # product_code 기준 중복 제거
df_unique.to_csv(output_file_path, index=False, encoding='utf-8')  # 2차 전처리 파일 저장

# medicines 테이블에 삽입할 SQL 쿼리 생성
df_medicines = pd.read_csv(output_file_path, encoding='utf-8', dtype={'product_code': str})

# product_code가 8자리인 경우 맨 앞에 0을 추가하여 9자리로 변환
df_medicines['product_code'] = df_medicines['product_code'].apply(
    lambda x: f"'{x.zfill(9)}'" if pd.notnull(x) and len(x) == 8 else f"'{x}'" if pd.notnull(x) else 'NULL'
)

# 각 컬럼값이 null인 경우 처리
df_medicines['category'] = df_medicines['category'].apply(lambda x: f"'{x}'" if pd.notnull(x) else 'NULL')
df_medicines['manufacturer'] = df_medicines['manufacturer'].apply(lambda x: f"'{x}'" if pd.notnull(x) else 'NULL')

# SQL 쿼리 생성
sql_statements = []
for index, row in df_medicines.iterrows():
    name = row['name'].replace("'", "''")  # 작은따옴표 이스케이프 처리
    product_code = row['product_code']
    category = row['category']
    manufacturer = row['manufacturer']
    sql = f"INSERT INTO medicines (name, product_code, category, manufacturer) " \
          f"VALUES ('{name}', {product_code}, {category}, {manufacturer});"
    sql_statements.append(sql)

# SQL 파일 생성 및 저장
with open(sql_file_path, 'w', encoding='utf-8') as f:
    for statement in sql_statements:
        f.write(statement + '\n')

# DUR

In [ ]:
import pandas as pd

# 파일 경로 설정
dur_file_path = '/Users/seungwook/Downloads/DUR.csv'  # 원본 DUR 파일
medicines_file_path = '/Users/seungwook/Downloads/약품코드_2차.csv'  # medicines 테이블의 CSV 파일 경로
sql_output_path = '/Users/seungwook/Downloads/dur_insert.sql'  # SQL 삽입 쿼리 경로
error_log_path = '/Users/seungwook/Downloads/error_log.txt'  # 오류 로그 파일 경로

# DUR 파일과 medicines 테이블 데이터를 읽어옴
df_dur = pd.read_csv(dur_file_path, encoding='euc-kr')
df_medicines = pd.read_csv(medicines_file_path, encoding='utf-8')

# a_product_code와 b_product_code가 8자리일 경우 9자리로 변환하는 방식 (f-string)
def format_code_to_9_digits(code):
    if isinstance(code, (int, float)):  # 숫자일 경우 처리
        code_str = f"{int(code):09d}"  # 9자리로 맞춰서 변환
    elif isinstance(code, str) and len(code) == 8:  # 8자리 문자열일 경우 앞에 '0' 추가
        code_str = f"0{code}"
    else:
        code_str = code  # 변환할 필요 없는 경우 그대로 유지
    return code_str

# a_product_code와 b_product_code 변환
df_dur['a_product_code'] = df_dur['a_product_code'].apply(format_code_to_9_digits)
df_dur['b_product_code'] = df_dur['b_product_code'].apply(format_code_to_9_digits)

# DUR 파일과 medicines 테이블 불일치 코드 추출 및 로그
df_medicines['product_code'] = df_medicines['product_code'].astype(str).str.strip()

# a_product_code와 b_product_code가 medicines 테이블에 없는 경우 로그 기록
invalid_a_codes = df_dur[~df_dur['a_product_code'].isin(df_medicines['product_code'])]
invalid_b_codes = df_dur[~df_dur['b_product_code'].isin(df_medicines['product_code'])]

with open(error_log_path, 'w', encoding='utf-8') as log_file:
    if not invalid_a_codes.empty:
        log_file.write("Invalid a_product_codes:\n")
        log_file.write(invalid_a_codes.to_string(index=False))
    if not invalid_b_codes.empty:
        log_file.write("\n\nInvalid b_product_codes:\n")
        log_file.write(invalid_b_codes.to_string(index=False))

# a_product_code와 b_product_code가 medicines 테이블에 있는 경우만 필터링
df_filtered_dur = df_dur[
    df_dur['a_product_code'].isin(df_medicines['product_code']) & 
    df_dur['b_product_code'].isin(df_medicines['product_code'])
]

# SQL 삽입문 생성
sql_statements = []
for _, row in df_filtered_dur.iterrows():
    a_product_code = row['a_product_code']
    b_product_code = row['b_product_code']
    reason = str(row['reason']).replace("'", "''") if pd.notnull(row['reason']) else 'NULL'
    etc = "'" + str(row['etc']).replace("'", "''") + "'" if pd.notnull(row['etc']) else 'NULL'
    
    sql = f"INSERT INTO dur (a_product_code, b_product_code, reason, etc) " \
          f"VALUES ('{a_product_code}', '{b_product_code}', '{reason}', {etc});"
    sql_statements.append(sql)

# SQL 파일에 작성
with open(sql_output_path, 'w', encoding='utf-8') as f:
    for statement in sql_statements:
        f.write(statement + '\n')


# clinic_location

In [3]:
import pandas as pd

clinic_data_path = '/Users/seungwook/Downloads/의료시설.csv'  # CSV 파일 경로
result_file = '/Users/seungwook/Downloads/clinic_location_insert.sql'  # SQL 파일 저장 경로

clinic_data = pd.read_csv(clinic_data_path, encoding='euc-kr')
def generate_sql(row):
    return f"""
    INSERT INTO clinic_location (district_name, type, name, inpatient_room, hospital_bed, tel, address, longitude, latitude)
    VALUES ('{row['district_name']}', '{row['type']}', '{row['name']}', {row['inpatient_room']}, {row['hospital_bed']}, '{row['tel']}', '{row['address']}', {row['longitude']}, {row['latitude']});
    """
sql_statements = [generate_sql(row) for index, row in clinic_data.iterrows()]
with open(result_file, 'w', encoding='utf-8') as f:
    f.write("\n".join(sql_statements))

In [5]:
import chardet

with open(dur_file_path, 'rb') as f:
    result = chardet.detect(f.read())
    print(result)

{'encoding': 'EUC-KR', 'confidence': 0.99, 'language': 'Korean'}
